**Author:** Benoît BAILLIF

**Purpose:** Find all available Pubchem CID for used compounds

**Input:**
- data/raw/
 - LincsID2FacilityID_LINCS_StandardizedCmpds_LSMIDs.txt : LINCS id to pert id mapping
 - CompoundTable_LINCS_StandardizedCmpds_LSMIDs.txt : LINCS id info, containing pubchem CID
- data/processed/
 - cmp_info_cmap : to find used compounds and most corresponding Pubchem CID

 
**Output:** 
- data/processed/
 - pert_id_to_pubchem_cid.csv : mapping used pert id to corresponding Pubchem CID

In [1]:
import time
import numpy     as np
import pandas    as pd
import pubchempy as pcp
from tqdm import tqdm_notebook as tqdm

In [2]:
%%capture
tqdm().pandas()

# Input

In [3]:
raw_data_directory = 'data/raw/'
processed_data_directory = 'data/processed/'

In [4]:
cmp_info_cmap_path = processed_data_directory + 'cmp_info_cmap.csv'
lincs_brd_link_path = raw_data_directory + 'LincsID2FacilityID_LINCS_StandardizedCmpds_LSMIDs.txt'
cmp_info_lincs_path = raw_data_directory + 'CompoundTable_LINCS_StandardizedCmpds_LSMIDs.txt'

# Output

In [5]:
pert_id_to_pubchem_cid_path = processed_data_directory + 'pert_id_to_pubchem_cid.csv'

# Main

In [6]:
cmp_info_cmap = pd.read_csv(cmp_info_cmap_path)
print(cmp_info_cmap.shape)
cmp_info_cmap.head()

(21220, 7)


,pert_id,pert_iname,is_touchstone,inchi_key,canonical_smiles,pubchem_cid,used_compound
0,BRD-A00100033,nifurtimox,1.0,ARFHIAQFJWUCFH-UHFFFAOYSA-N,CC1CS(=O)(=O)CCN1N=Cc1ccc([N+](=O)[O-])o1,6842999.0,1
1,BRD-A00150179,5-hydroxytryptophan,0.0,QSHLMQDRPXXYEE-UHFFFAOYSA-N,NC(Cc1c[nH]c2cccc(O)c12)C(=O)O,589768.0,0
2,BRD-A00267231,hemado,1.0,KOCIMZNSNPOGOP-UHFFFAOYSA-N,CCCCC#Cc1nc(NC)c2ncn(C3OC(CO)C(O)C3O)c2n1,4043357.0,1
3,BRD-A00420644,SA-3676,0.0,ASCBUEVCEVGOFP-UHFFFAOYSA-N,CCN1c2ccccc2NC2N=C(OC)C(c3ccccc3)C21,2853908.0,1
4,BRD-A00474148,BRD-A00474148,0.0,RCGAUPRLRFZAMS-UHFFFAOYSA-N,O=C1Cc2cc([S+](=O)([O-])N3CCN(c4ccc(O)cc4)CC3)...,44825297.0,1


In [7]:
used_cmp_info_cmap = cmp_info_cmap[cmp_info_cmap['used_compound'] == 1]
n_used_compounds = used_cmp_info_cmap.shape[0]
print('Number of used compounds: ' + str(n_used_compounds))

Number of used compounds: 9035


In [8]:
lincs_brd_link = pd.read_csv(lincs_brd_link_path, sep='\t')
lincs_brd_link.head()

,SM_LINCS_ID,SM_Center_Canonical_ID
0,LSM-1001,10001
1,LSM-1001,BRD-K07691486
2,LSM-1001,PCCSE_Roscovitine
3,LSM-1002,10002
4,LSM-1002,BRD-K68191783


In [9]:
cmp_info_lincs = pd.read_csv(cmp_info_lincs_path, sep='\t')
cmp_info_lincs.head()

,SM_LINCS_ID,SM_PubChem_CID,SM_SMILES_Parent,SM_InChI_Parent,SM_InChI_Key_Parent,SM_Molecular_Mass
0,LSM-1001,160355.0,CC[C@H](CO)Nc1nc(NCc2ccccc2)c3ncn(C(C)C)c3n1,InChI=1S/C19H26N6O/c1-4-15(11-26)22-19-23-17(2...,BTIHMVBBUGXLCJ-OAHLLOKOSA-N,354.22
1,LSM-1002,24880028.0,Cc1cn(cn1)c2cc(cc(c2)C(F)(F)F)C(=O)Nc3ccc(C)c(...,InChI=1S/C23H18F3N5O3/c1-13-3-4-17(10-19(13)30...,IYUFHBXMTTXZBE-UHFFFAOYSA-N,469.14
2,LSM-1003,24875320.0,Cc1ccc(cc1Nc2cncc(c2)C(=O)N)C(=O)Nc3cccc(c3)C(...,InChI=1S/C21H17F3N4O2/c1-12-5-6-13(8-18(12)27-...,SAAYRHKJHDIDPH-UHFFFAOYSA-N,414.13
3,LSM-1004,11338033.0,Clc1cccc(Cl)c1C(=O)Nc2c[nH]nc2C(=O)NC3CCNCC3,InChI=1S/C16H17Cl2N5O2/c17-10-2-1-3-11(18)13(1...,OVPNQJVDAFNBDN-UHFFFAOYSA-N,381.08
4,LSM-1005,9911830.0,COc1cc2nccc(Oc3ccc(NC(=O)Nc4cc(C)on4)c(Cl)c3)c...,InChI=1S/C22H19ClN4O5/c1-12-8-21(27-32-12)26-2...,SPMVMDHWKHCIDT-UHFFFAOYSA-N,454.10


In [10]:
used_cmp_info_LINCS = cmp_info_lincs.merge(lincs_brd_link, on='SM_LINCS_ID')
used_cmp_info = used_cmp_info_cmap.merge(used_cmp_info_LINCS, left_on='pert_id', right_on='SM_Center_Canonical_ID')

used_cmp_info.head()

,pert_id,pert_iname,is_touchstone,inchi_key,canonical_smiles,pubchem_cid,used_compound,SM_LINCS_ID,SM_PubChem_CID,SM_SMILES_Parent,SM_InChI_Parent,SM_InChI_Key_Parent,SM_Molecular_Mass,SM_Center_Canonical_ID
0,BRD-A00100033,nifurtimox,1.0,ARFHIAQFJWUCFH-UHFFFAOYSA-N,CC1CS(=O)(=O)CCN1N=Cc1ccc([N+](=O)[O-])o1,6842999.0,1,LSM-1232,31772.0,CC1CS(=O)(=O)CCN1N=Cc2oc(cc2)[N+](=O)[O-],"InChI=1S/C10H13N3O5S/c1-8-7-19(16,17)5-4-12(8)...",ARFHIAQFJWUCFH-UHFFFAOYSA-N,287.06,BRD-A00100033
1,BRD-A00267231,hemado,1.0,KOCIMZNSNPOGOP-UHFFFAOYSA-N,CCCCC#Cc1nc(NC)c2ncn(C3OC(CO)C(O)C3O)c2n1,4043357.0,1,LSM-1233,4043357.0,CCCCC#Cc1nc(NC)c2ncn(C3OC(CO)C(O)C3O)c2n1,InChI=1S/C17H23N5O4/c1-3-4-5-6-7-11-20-15(18-2...,KOCIMZNSNPOGOP-UHFFFAOYSA-N,361.18,BRD-A00267231
2,BRD-A00420644,SA-3676,0.0,ASCBUEVCEVGOFP-UHFFFAOYSA-N,CCN1c2ccccc2NC2N=C(OC)C(c3ccccc3)C21,2853908.0,1,LSM-6366,2853908.0,CCN1C2C(Nc3ccccc13)N=C(OC)C2c4ccccc4,InChI=1S/C19H21N3O/c1-3-22-15-12-8-7-11-14(15)...,ASCBUEVCEVGOFP-UHFFFAOYSA-N,307.17,BRD-A00420644
3,BRD-A00474148,BRD-A00474148,0.0,RCGAUPRLRFZAMS-UHFFFAOYSA-N,O=C1Cc2cc([S+](=O)([O-])N3CCN(c4ccc(O)cc4)CC3)...,44825297.0,1,LSM-1234,44825297.0,Oc1ccc(cc1)N2CCN(CC2)S(=O)(=O)c3ccc4NC(=O)Cc4c3,InChI=1S/C18H19N3O4S/c22-15-3-1-14(2-4-15)20-7...,YCOCADXVPASJAM-UHFFFAOYSA-N,373.11,BRD-A00474148
4,BRD-A00520476,AF-DX-116,1.0,UBRKDAVQCKZSPO-UHFFFAOYSA-N,CCN(CC)CC1CCCCN1CC(=O)N1c2ccccc2C(=O)Nc2cccnc21,107867.0,1,LSM-22625,107867.0,CCN(CC)CC1CCCCN1CC(=O)N2c3ccccc3C(=O)Nc4cccnc24,InChI=1S/C24H31N5O2/c1-3-27(4-2)16-18-10-7-8-1...,UBRKDAVQCKZSPO-UHFFFAOYSA-N,421.25,BRD-A00520476


In [11]:
n_LINCS_and_used_compounds = used_cmp_info.shape[0]
print('Number of common compounds between LINCS and used compounds: ' + str(n_LINCS_and_used_compounds))
print('So there are ' + str(n_used_compounds - n_LINCS_and_used_compounds) + ' more compounds in total than LINCS')

Number of common compounds between LINCS and used compounds: 9000
So there are 35 more compounds in total than LINCS


In [12]:
# Difference compared to paper version : I used the first table, 
# and not the one with LINCS data joined (that removed rows)
L1000_cids = used_cmp_info_cmap[['pert_id', 'pubchem_cid']]

L1000_cids = L1000_cids.dropna()

# Difference compared to paper version : I removed only -666
L1000_cids = L1000_cids[L1000_cids['pubchem_cid'] > 0]

L1000_cids.head()

,pert_id,pubchem_cid
0,BRD-A00100033,6842999.0
2,BRD-A00267231,4043357.0
3,BRD-A00420644,2853908.0
4,BRD-A00474148,44825297.0
5,BRD-A00520476,107867.0


In [13]:
LINCS_cids = used_cmp_info[['pert_id', 'SM_PubChem_CID']]
LINCS_cids = LINCS_cids.dropna()
LINCS_cids = LINCS_cids.rename({'SM_PubChem_CID' : 'pubchem_cid'}, axis=1)
LINCS_cids.head()

,pert_id,pubchem_cid
0,BRD-A00100033,31772.0
1,BRD-A00267231,4043357.0
2,BRD-A00420644,2853908.0
3,BRD-A00474148,44825297.0
4,BRD-A00520476,107867.0


In [14]:
def fetch_cids(input_str, input_type='smiles') :
    """Fetch Pubchem CIDs for a given input string.
    
    Args:
        input_str  (str) : input string to consider
        input_type (str) : type of the input (e.g. smiles, inchikey...)
        
    Returns:
        list : Pubchem CIDs
        
    """
    cids = None
    max_retries = 5
    current_try = 0
    while cids == None and current_try < max_retries:
        try :
            cids = pcp.get_cids(input_str, input_type)
            print(input_str)
        except : 
            print('Retry on ' + input_str)
            current_try = current_try + 1
        
    return cids

In [15]:
time_start = time.time()

cids_from_smiles = used_cmp_info_cmap['canonical_smiles'].dropna().astype(str).apply(fetch_cids)

print('Fetch CID for all compounds : Time elapsed : {} seconds'.format(time.time() - time_start))

CC1CS(=O)(=O)CCN1N=Cc1ccc([N+](=O)[O-])o1
CCCCC#Cc1nc(NC)c2ncn(C3OC(CO)C(O)C3O)c2n1
CCN1c2ccccc2NC2N=C(OC)C(c3ccccc3)C21
O=C1Cc2cc([S+](=O)([O-])N3CCN(c4ccc(O)cc4)CC3)ccc2N1
CCN(CC)CC1CCCCN1CC(=O)N1c2ccccc2C(=O)Nc2cccnc21
OC(CCN1CCCCC1)(c1ccccc1)C1CC2C=CC1C2
COc1ccc(C(CC(=O)N2CCCC(C)C2)c2c(O)cc(OC)cc2OC)cc1
C#C[C@]1(O)CCC2C3CCC4=C(CCC(=O)C4)C3CC[C@@]21C
Cn1c(=O)c2c(ncn2CC(O)CO)n(C)c1=O
C=CCc1ccccc1OCC(O)CNC(C)C
CCC(c1ccc(O)cc1)C(CC)c1ccc(O)cc1
O=c1ncccn1[C@@H]1O[C@H](CO)[C@H](O)C1O
CCCN(C/C=C/I)C1CCc2ccc(O)cc2C1
CC(O)[C@H](O)C1=Nc2c(nc(N)[nH]c2=O)NC1
OCc1cc(C(O)CNCCCCCCOCCCCc2ccccc2)ccc1O
C[C@@H]1CC2C3C[C@H](F)C4=CC(=O)C=C[C@]4(C)[C@@]3(F)[C@@H](O)C[C@]2(C)[C@@]1(O)C(=O)CO
CC(=O)O[C@]1(C(C)=O)CCC2C3C=C(Cl)C4=CC(=O)CC[C@]4(C)C3CC[C@@]21C
CCCCN1CCCCC1C(=O)Nc1c(C)cccc1C
CC(=O)OCC(=O)[C@@]1(O)CCC2C3CCC4=CC(=O)C=CC4(C)C3[C@@H](O)C[C@@]21C
COc1ccccc1N1CCN(CC(O)COc2cccc3ccccc23)CC1
CN(CCO)CC(O)Cn1cnc2c1c(=O)n(C)c(=O)n2C
COc1ccc(C2C(C(=O)O)c3ccccc3C(=O)N2Cc2ccc(Cl)cc2)cc1
CCC(=O)NC(c1ccco1)c1c

CCn1cc(CN2CCCC(C(=O)c3cccc(C(F)(F)F)c3)C2)cn1
CC1OC(OC2CCC3(C)C(CCC4(C)C3CC=C3C5CC(C)(C)CCC5(C(=O)OC5OC(CO)C(OC6OCC(OC7OC(CO)C(O)C(O)C7O)C(OC7OCC(O)C(O)C7O)C6O)C(O)C5O)CCC34C)C2(C)C)C(O)C(O)C1OC1OCC(O)C(O)C1O
CC1CC(=O)C2C(O)(C1)CC(O)C13OC1c1c(O)cccc1C(=O)C23O
Cc1onc(-c2c(F)cccc2Cl)c1C(=O)N[C@@H]1C(=O)N2C(C(=O)O)C(C)(C)S[C@H]12
C=CC[N@@+]12CC[C@]34c5ccccc5N5C=C6C7N(C=C(C53)[C@@H](C[C@@H]41)C(=CCO)C2)c1ccccc1[C@@]71CC[N@@+]2(CC=C)CC(=CCO)[C@H]6C[C@H]12
CCOC(=O)C1=C(N)Oc2ccc(Br)cc2C1C(C#N)C(=O)OCC
COc1cc2c(cc1O)C(c1cccs1)N(Cc1ccc([N+](=O)[O-])o1)CC2
CC1=C(CC(=O)O)c2cc(F)ccc2/C1=C\c1ccc(S(C)=O)cc1
OC12C3C4CC5C6C4C1C6C(C53)N2CCc1cccc(F)c1
C[C@H]1O[C@@H](O[C@H]2CC[C@@]3(C)C(CC[C@]4(C)C3C=C[C@]35OC[C@@]6(CCC(C)(C)C[C@H]63)[C@@H](O)C[C@]54C)[C@]2(C)CO)[C@H](O[C@@H]2O[C@H](CO)[C@@H](O)[C@H](O)[C@H]2O)[C@@H](O[C@@H]2O[C@H](CO)[C@@H](O[C@@H]3O[C@H](CO)[C@@H](O)[C@H](O)[C@H]3O)[C@H](O)[C@H]2O)[C@H]1O
CC[N+](C)(CC)CCOC(=O)C(O)(c1ccccc1)C1CCCCC1
NCC(O)c1ccc(O)cc1
CN1CCCCC1CCN1c2ccccc2Sc2ccc(S(C)=O)c

CC(=O)OC1C(=O)C2(C)C(O)CC3OCC3(OC(C)=O)C2C(OC(=O)c2ccccc2)C2(O)CC(OC(=O)C(O)C(NC(=O)c3ccccc3)c3ccccc3)C(C)=C1C2(C)C
CO[C@H]1C[C@@H]2CC[C@@H](C)[C@@](O)(O2)C(=O)C(=O)N2CCCC[C@H]2C(=O)O[C@H](C(C)CC2CC[C@@H](O)[C@H](OC)C2)CC(=O)[C@H](C)/C=C(\C)[C@@H](O)[C@@H](OC)C(=O)[C@@H](C)C[C@H](C)/C=C/C=C/C=C\1C
CCC(C)N1CCC2(CC1)N=C1C(=C3NC(=O)C(C)=CC=C[C@H](C)[C@H](O)[C@@H](C)[C@@H](O)[C@@H](C)[C@H](OC(C)=O)[C@H](C)[C@@H](OC)C=CO[C@@]4(C)Oc5c(C)c(O)c(c1c5C4=O)C3=O)N2
CC(=O)NC(c1cccs1)c1cc(F)c2cccnc2c1O
CCN1CCN(C(c2cccnc2)c2cc(Cl)c3cccnc3c2O)CC1
NCC(CS(=O)(=O)O)c1ccc(Cl)cc1
CC(C(O)c1ccc(O)cc1)N1CCC(Cc2ccccc2)CC1
CC(C)c1cn(C2OC(COC(=O)c3ccccc3)C(OC(=O)c3ccccc3)C2OC(=O)c2ccccc2)c(=O)[nH]c1=O
CC1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23
CC1CCC[C@H](O)CCCCCc2cc(O)cc(O)c2C(=O)O1
CC1C(=O)C(=O)C=C2C1=CC=C1[C@@]2(C)CCC2(C)[C@@H]3C[C@](C)(C(=O)O)CC[C@]3(C)CC[C@]12C
CC1CCc2c(N3CCC(O)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23
CCCC[N+]1(C)[C@H]2C[C@H](OC(=O)[C@H](CO)c3ccccc3)C[C@@H]1[C@@H]1O[C@H]12
O=C1c2ccccc2CCC1CNCC

COC(c1ccccc1)(c1ccccc1)C(Oc1nc(C)cc(C)n1)C(=O)O
CC(C)(C)NCC(O)COc1cccc2[nH]c(=O)[nH]c12
CC1CC(OC(=O)c2ccccc2O)CC(C)(C)C1
C[C@@H]1O[C@@H](O[C@@H]2C=C3CCC4C(CC[C@]5(C)[C@@H](c6ccc(=O)oc6)CC[C@]45O)[C@@]3(C)CC2)[C@H](O)[C@H](O)[C@H]1O
CCC(C)n1ncn(-c2ccc(N3CCN(c4ccc(OCC5COC(Cn6cncn6)(c6ccc(Cl)cc6Cl)O5)cc4)CC3)cc2)c1=O
CC1OC(OCC2OC(Oc3c(-c4ccc(O)c(O)c4)oc4cc(O)cc(O)c4c3=O)C(O)C(O)C2O)C(O)C(O)C1O
Cc1cccc(N2CCN(C(=O)COc3noc4nc(C)cc(C)c34)CC2C)c1
O=C1CC(c2ccc(O)c(O)c2)Oc2cc(O)cc(O)c21
CC1CC2C3CCC4=CC(=O)C=CC4(C)C3(F)C(O)CC2(C)C1(O)C(=O)CO
OC1c2c(F)cccc2N=C2CCCCN21
CCNC(=O)C1OC(n2cnc3c(N)ncnc32)C(O)C1O
COC(=O)C(CCCNC(=N)N)NC(=O)CCCc1c(-c2ccccc2)[nH]c2c(F)cc(F)cc12
COC(=O)C1C(C)=NC(C)=C(C(=O)O[C@@H]2CCCN(Cc3ccccc3)C2)[C@@H]1c1cccc([N+](=O)[O-])c1
COc1cc([C@@H]2c3cc4c(cc3C(O[C@@H]3O[C@@H]5CO[C@@H](c6cccs6)OC5[C@H](O)[C@H]3O)C3COC(=O)[C@@H]32)OCO4)cc(OC)c1O
O=C1CSC(c2ccccc2Cl)c2cc(Cl)ccc2N1
O=C1OC(CN2CCOCC2)CN1N=Cc1ccc([N+](=O)[O-])o1
C=C(C(=O)O[C@H]1C/C(C)=C/CC/C(CO)=C/[C@H]2OC(=O)C(=C)[C@H]12)C(

CNC(=O)Nc1ccc(-c2nc(N3CC4CCC(C3)O4)c3cnn(C4CCC5(CC4)OCCO5)c3n2)cc1
NC(Cn1ccc(=O)n(Cc2ccccc2C(=O)O)c1=O)C(=O)O
CC(=O)Nc1c(I)c(C(=O)N[C@H]2C(O)O[C@H](CO)[C@@H](O)[C@@H]2O)c(I)c(N(C)C(C)=O)c1I
CCOC(=O)C1=C(c2ccccc2)N=c2sc(=Cc3cc(OC)c(OC)c(OC)c3)c(=O)n2C1c1cccs1
CC#CCC(C)[C@H](O)/C=C/[C@@H]1[C@H]2C/C(=C/CCCC(=O)O)C[C@H]2C[C@H]1O
CCN(CC)CCCC(C)Nc1c2ccc(Cl)cc2nc2ccc(OC)cc12
CCN1CC(CCN2CCOCC2)C(c2ccccc2)(c2ccccc2)C1=O
CC12CCC(=O)C=C1CCC1C2[C@@H](O)CC2(C)C1CC[C@]2(O)C(=O)COP(=O)(O)O
c1ccc(C2CN3CCSC3=N2)cc1
CC1OC(OC2CC(O)C3(CO)C4C(O)CC5(C)C(C6=CC(=O)OC6)CCC5(O)C4CCC3(O)C2)C(O)C(O)C1O
C/C=C1\C2C=C(C)CC1(N)c1ccc(=O)[nH]c1C2
CC1(C)C(=O)CC[C@@]2(C)C1C[C@@H](O)[C@@]1(C)C3=CC(=O)O[C@@H](c4ccoc4)[C@]3(C)CCC21
Oc1ccc2c3c1O[C@H]1c4[nH]c5ccccc5c4C[C@@]4(O)C(C2)N(CC2CC2)CC[C@]314
CNCC(O)c1ccc(OC(=O)C(C)(C)C)c(OC(=O)C(C)(C)C)c1
COC[C@H]([C@@H](O)[C@@H](O)C(=O)NCC[C@@H](C)c1nc(/C=C/C[C@H]2OC3(C[C@H](O)[C@H]2C)OC([C@@H](C[C@@H](O)[C@@H](C)[C@@H](O)[C@@H](C)/C=C(C)/C(C)=C/C=C/C(C)=C\C#N)OC)[C@@H](OP(=O)(O)O)C

...

C[C@]12CCC3C(C1[C@@H]1C[C@@H]1[C@@]21CCC(=O)O1)[C@H]1C[C@H]1C1=CC(=O)CC[C@@]13C
N=C(N)NCCC[C@H](N)C(=O)NCC(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](CO)C(=O)O
C[C@]12CC[C@@H]3[C@@H](CC[C@H]4NC(=O)C=C[C@@]43C)[C@@H]1CC[C@H]2C(=O)Nc1cc(C(F)(F)F)ccc1C(F)(F)F
COc1cc(/C=C2\CCCN([C@@H](C)c3ccc(F)cc3)C2=O)ccc1-n1cnc(C)c1
CC(C)c1cc2c(cc1S(=O)(=O)O)[C@@]1(C)CCC[C@@](C)(C(=O)O)[C@@H]1CC2
CN1CCc2nc(C(=O)N[C@H]3C[C@H](C(=O)N(C)C)CC[C@@H]3NC(=O)C(=O)Nc3ccc(Cl)cn3)sc2C1
CCc1cn([C@H]2C[C@H](O)[C@@H](CO)O2)c(=O)[nH]c1=O
Cc1cc(C)cc(NC(=O)Cc2ccc(OC(C)(C)C(=O)O)cc2)c1
Cc1cc(Oc2ccc3nc(COc4ccc(CC5SC(=O)NC5=O)cc4)n(C)c3c2)cc(C)c1N
NCCCC(N)(C(=O)O)C(F)F
COc1cc2c(cc1OC)CN(CCc1ccc(NC(=O)c3cccc4c(=O)c5cccc(OC)c5[nH]c34)cc1)CC2
CN1CCC(COc2cnc(-c3cccc(Cn4nc(-c5cccc(C#N)c5)ccc4=O)c3)nc2)CC1
COc1ccc(C(=O)N2CCCc3cc(C4=NNC(=O)SC4C)ccc32)cc1OC
OC[C@H]1O[C@@H](c2ccc(Cl)c(Cc3ccc(O[C@H]4CCOC4)cc3)c2)[C@H](O)[C@@H](O)[C@@H]1O
Nc1nc(=O)n([C@@H]2CS[C@H](CO)O2)cc1F
CC1(C)[C@@H](O)CC[C@]2(C)[C@@H]3C(=O)C=C4[C@H]5C[C@@](C)(C(=O)O)CC[C@]

CC(C)c1ccc2c(c1)C(N1CCN(CCO)CC1)Cc1ccc(F)cc1S2
C[C@]12CC[C@H]3[C@@H](CC(=O)[C@H]4C/C(=N/OCCN)CC[C@@]43C)[C@@H]1CCC2=O
CCC(C)n1ncn(-c2ccc(N3CCN(c4ccc(OC[C@H]5CO[C@](Cn6cncn6)(c6ccc(Cl)cc6Cl)O5)cc4)CC3)cc2)c1=O
COc1cc2c(cc1OC)CC(=O)N(CCCN(C)C[C@@H]1Cc3cc(OC)c(OC)cc31)CC2
CC(C)(C)c1cc(C(C)(C)C)c(NC(=O)c2c[nH]c3ccccc3c2=O)cc1O
C/C(=C\c1csc(C)n1)[C@@H]1C[C@@H]2O[C@]2(C)CCC[C@H](C)[C@H](O)[C@@H](C)C(=O)C(C)(C)[C@@H](O)CC(=O)N1
CC(C)C[C@H](NC(=O)CNC(=O)c1cc(Cl)ccc1Cl)B1OC(=O)CC(CC(=O)O)(C(=O)O)O1
N#Cc1c(NC(=O)c2cccc3ccccc23)sc2c1CCCC2
CO[C@@H]1[C@@H](O[C@@H]2O[C@H](C)[C@@H](O[C@H]3C[C@@](C)(OC(=O)CC(C)C)[C@@H](O)[C@H](C)O3)[C@H](N(C)C)[C@H]2O)[C@@H](CC=O)C[C@@H](C)[C@@H](O)/C=C/C=C/C[C@@H](C)OC(=O)C[C@H]1OC(C)=O
Cc1sc2c(c1C)C(c1ccc(Cl)cc1)=N[C@H](CC(=O)OC(C)(C)C)c1nnc(C)n1-2
Cc1sc2c(c1C)C(c1ccc(Cl)cc1)=N[C@@H](CC(=O)OC(C)(C)C)c1nnc(C)n1-2
COc1ccc(C2(CNC(=O)c3ccc(NC(=O)c4ccco4)cc3)CCOCC2)cc1
O=c1c(-c2ccc(O)cc2)coc2c(C3OC(CO)C(O)C(O)C3O)c(O)ccc12
OC1O[C@@H]2COC(c3ccccc3)O[C@H]2[C@H](O)[C@H]1O
C

Nc1ncnc2c1c(-c1cccc(OCc3ccccc3)c1)cn2C1CC(CN2CCC2)C1
CCNC(=O)C1=C(c2ccc(CN3CCOCC3)cc2)C(=C2C=C(C(C)C)C(O)=CC2=O)ON1
Cc1ccc(C(=O)Nc2cccc(C(F)(F)F)c2)cc1Nc1nc(-c2cccnc2)nc2c1cnn2C
N#Cc1ccc(NCCNCC(=O)N2CCC[C@H]2C#N)nc1
CNC(=O)c1ccccc1Nc1nc(Nc2ccc(N3CCOCC3)cc2OC)ncc1Cl
CC(C)(C)/[N+]([O-])=C/c1ccc(S(=O)(=O)O)cc1S(=O)(=O)O
Nc1nc(OCc2ccccc2)c2[nH]cnc2n1
CC(C)c1ccc(C(=O)NCCOc2ccc(C(=O)O)cc2)cc1
CC(C)(F)C[C@H](N[C@@H](c1ccc(-c2ccc(S(C)(=O)=O)cc2)cc1)C(F)(F)F)C(=O)NC1(C#N)CC1
CS(=O)(=O)Nc1ccc2oc3c(OC(F)F)ccc(C(=O)Nc4c(Cl)cncc4Cl)c3c2c1
Cc1[nH]c(=O)c(C#N)cc1-c1ccc2nccn2c1
COC(=O)N1CCN(Cc2cccc(NC(=O)Nc3ccc(C)nc3)c2F)CC1
COC[C@H](NC(=O)c1cnc(C)s1)C(=O)N[C@@H](COC)C(=O)N[C@@H](Cc1ccccc1)C(=O)C1(C)CO1
CC(C)C[C@H](N[C@@H](Cc1cncn1Cc1cc(Cl)cc(Cl)c1)C(=O)O)C(=O)O
CC1(CN)Cc2ccccc2C1
CNC(=O)c1ccc2cc([C@]3(O)CCn4cncc43)ccc2c1
CCC(CC)O[C@@H]1C=C(C(=O)O)C[C@H](N)[C@H]1NC(C)=O
COc1cccc2c1=NC(=c1nc(C3CCC(C(=O)O)CC3)n3c1=C(N)N=CN3)C=2
C=CC(=O)Nc1cc(Nc2nccc(-c3cn(C)c4ccccc34)n2)c(OC)cc1N(C)CCN(C)C
C[C@](O)(COc1c

NC(=O)C(c1ccccc1)(c1ccc(F)cc1)c1ccc(F)cc1
C=C1/C(=C\C=C2/CCC[C@@]3(C)[C@H]2CC[C@@H]3[C@H](C)/C=C/C=C/C(O)(CC)CC)C[C@@H](O)C[C@@H]1O
O=C(O)c1cc2occc2[nH]1
CN[C@@H]1CC[C@@H](c2ccc(Cl)c(Cl)c2)c2ccccc21
CN1CCC2=C(C1)c1ccccc1Cc1ccccc12
CN1CCC(CNc2ccc3ncc(-c4cccc(OC(F)(F)F)c4)n3n2)CC1
COc1ccc2c(O[C@H]3CC4C(=O)N(C)CCCC/C=C\[C@@H]5C[C@@]5(C(=O)NS(=O)(=O)C5CC5)NC(=O)[C@@H]4C3)cc(-c3nc(C(C)C)cs3)nc2c1C
Fc1ccc(-n2cc(CCCCN3CCC4(CC3)OCc3ccccc34)c3ccccc32)cc1
C=CCN1CC[C@]2(C)c3cc(O)ccc3CC1[C@@H]2C
CCc1c2c(nc3ccc(O)cc13)-c1cc3c(c(=O)n1C2)COC(=O)[C@]3(O)CC
CC1(C)CC(=O)c2c(C(F)(F)F)nn(-c3ccc(C(N)=O)c(N[C@H]4CC[C@H](O)CC4)c3)c2C1
Cc1cc(OCC(=O)O)cc(C)c1Cc1ccc(O)c(C(C)C)c1
Cc1cc(S(=O)(=O)O)c2c(C)ccc(C(C)C)cc1-2
Cc1c(S(=O)(=O)O)oc2c1C(=O)C(=O)c1c-2ccc2c1CCCC2(C)C
CC(C)=CCOc1ccc(/C=C/C(=O)c2ccc(OCC=C(C)C)cc2OCC(=O)O)cc1
O=C(O[C@H]1CN2CCC1CC2)N1CCc2ccccc2[C@@H]1c1ccccc1
O=C1NC(=O)C2(CCOc3ccc(F)cc32)N1
CN1CCN(c2nc(C3=C(c4c[nH]c5ccccc45)C(=O)NC3=O)c3ccccc3n2)CC1
CC1CN(c2c(F)c(N)c3c(=O)c(C(=O)O)cn(C4CC4)c3c2F)C

CC(C)OC(=O)C1=CN(C(=O)c2ccc(F)c(F)c2)CC(C)(C)c2c1[nH]c1ccccc21
C=CC(=O)Nc1ccc(CN(C(=O)Nc2c(Cl)c(OC)cc(OC)c2Cl)c2cc(Nc3ccc(N4CCN(C)CC4)cc3)ncn2)cc1
CCN(Cc1ccccc1)C(=O)Cn1c(=O)n(C)c2cnc(-c3ccccc3)nc21
CC[C@@H](C)Nc1cc(C(=O)NC2CC3CCC(C2)N3c2ccc(C(=O)C3CC3)cn2)c(C)cc1C(N)=O
CN(C)C/C=C/C(=O)Nc1cccc(Nc2ncc3c(n2)N(C)c2ccccc2C(=O)N3C)c1
COc1cc(C(=O)NC2CCN(C)CC2)ccc1Nc1ncc2c(n1)N(C1CCCC1)CCC(=O)N2C(C)C
O=C(CC(c1ccccc1)c1ccccc1)N1CCN(C(c2ccccc2)c2ccccc2)CC1
COCCNC(=O)COc1ccc(OCCNC[C@H](O)COc2ccccc2)cc1
Cc1cn(C2CC(N=[N+]=[N-])C(CO)O2)c(=O)[nH]c1=O
O=c1[nH]c2cc(C(F)(F)F)c(N3CCOCC3)cc2n(CP(=O)(O)O)c1=O
C[C@@H]1CN(Cc2ccc(F)cc2)CCN1C(=O)COc1ccc(Cl)cc1NC(N)=O
C[C@H](CSC(=O)c1ccccc1)C(=O)N1C[C@@H](Sc2ccccc2)C[C@H]1C(=O)O
Fetch CID for all compounds : Time elapsed : 8079.902158498764 seconds


In [34]:
cids_from_smiles

0           [31772]
2         [4043357]
3         [2853908]
4        [44825297]
5          [107867]
            ...    
21198      [121877]
21199        [5726]
21200      [208953]
21201      [512282]
21202       [92400]
Name: canonical_smiles, Length: 9035, dtype: object

In [57]:
indexes_incorrect_smiles = []
for i in cids_from_smiles.index :
    if cids_from_smiles[i] == None :
        print('Incorrect SMILES : ' + used_cmp_info_cmap.loc[i, ]['canonical_smiles'])
        indexes_incorrect_smiles.append(i)

Incorrect SMILES : CCP(CC)(CC)=[Au]SC1CC(OC(C)=O)C(OC(C)=O)C(OC(C)=O)C1OC(C)=O


In [61]:
used_cmp_info_cmap['pubchem_cid_from_smiles'] = cids_from_smiles.drop(indexes_incorrect_smiles).apply(lambda l : '' if len(l) == 0 else float(l[0]))

C:\Users\Ben\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
used_pert_id_cid = used_cmp_info_cmap[['pert_id', 'pubchem_cid_from_smiles']]
used_pert_id_cid = used_pert_id_cid.rename({'pubchem_cid_from_smiles' : 'pubchem_cid'}, axis=1)
used_pert_id_cid = used_pert_id_cid[used_pert_id_cid != 0]
used_pert_id_cid.head()

,pert_id,pubchem_cid
0,BRD-A00100033,31772.0
2,BRD-A00267231,4043357.0
3,BRD-A00420644,2853908.0
4,BRD-A00474148,44825297.0
5,BRD-A00520476,107867.0


In [78]:
used_pert_id_cid.to_csv(raw_data_directory + 'cid_from_smiles.csv')

In [82]:
pert_id_to_pubchem_cid = pd.concat([L1000_cids, LINCS_cids, used_pert_id_cid])
pert_id_to_pubchem_cid = pert_id_to_pubchem_cid.drop_duplicates()
pert_id_to_pubchem_cid = pert_id_to_pubchem_cid.dropna()
pert_id_to_pubchem_cid['pubchem_cid'] = pert_id_to_pubchem_cid['pubchem_cid'].astype(int)

In [83]:
pert_id_to_pubchem_cid.to_csv(pert_id_to_pubchem_cid_path, index=False)